<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Lesson 6.02 - Bootstrapping and Bagging

## Importing libraries


We'll need the following libaries for today's lecture:
- `pandas`
- `numpy`
- `DecisionTreeClassifier` from `sklearn`'s `tree` module
- `BaggingClassifier` from `sklearn`'s `ensemble` module
- `train_test_split` and `cross_val_score` from `sklearn`'s `model_selection` module
- `accuracy_score` from `sklearn`'s `metrics` module

In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score

## Load the Data

We'll be using the `Heart.csv` from the [ISLR Website](https://www-bcf.usc.edu/~gareth/ISL/). There's a copy in this repo under `./datasets/Heart.csv`.

In [ ]:
# Read in the Heart .csv data.
df = pd.read_csv('./datasets/Heart.csv')

# Drop the `Unnamed: 0` column.


In [ ]:
# Check the first few rows to make sure we dropped the column properly.


## Data cleaning: Drop rows with null values

In [ ]:
# Check the shape of the data.


In [ ]:
# How much missing data do we have?


In [ ]:
# Drop NAs.


In [ ]:
# Confirm all missing data is dropped.


In [ ]:
# What's the shape of our data now?


## Feature Engineering

In [ ]:
# Create dummies for the `ChestPain`, `Thal`, and `AHD` columns.
# Be sure to set `drop_first=True`.
df = 

# Confirm we did this correctly.
df.head()

## Model Prep: Create `X` and `y` variables

Our target column will be `AHD_Yes`: 
- 1 means the patient has heart disease
- 0 means they aren't diagnosed with heart disease

In [ ]:

X = df.drop('AHD_Yes', axis='columns')

y = df['AHD_Yes']

In [ ]:
# What is the accuracy of our baseline model?


<details><summary>What does a false positive mean in this case?</summary>
    
- A false positive indicates someone **falsely** predict as being in the **positive** class.
- This is someone we incorrectly think has heart disease.
- Incorrectly predicting someone to have heart disease is bad... but it _might_ be worse to incorrectly predict that someone is healthy!
</details>

## Model Prep: Train/Test Split

In [ ]:
# Split data into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=42,
                                                    stratify=y)

## Model Instantiation

In [ ]:
# Instantiate `DecisionTreeClassifier` object.
tree = 

## Model Evaluation

In [ ]:
# Get a cross_val_score for our tree.


In [ ]:
# Fit and score on the training data.


In [ ]:
# Score on the testing data.


<details><summary> Where do decision trees tend to fall on the Bias/Variance spectrum?</summary>
    
- Decision trees very easily overfit.
- They tend to suffer from **high error due to variance**.
</details>

## Introduction to Ensemble Methods
We can list out the different types of models we've built thus far:
- Linear Regression
- Logistic Regression
- $k$-Nearest Neighbors
- Naive Bayes Classification _(maybe)_

If we want to use any of these models, we follow the same type of process.
1. Based on our problem, we identify which model to use. (Is our problem classification or regression? Do we want an interpretable model?)
2. Fit the model using the training data.
3. Use the fit model to generate predictions.
4. Evaluate our model's performance and, if necessary, return to step 2 and make changes.

So far, we've always had exactly one model. Today, however, we're going to talk about **ensemble methods**. Mentally, you should think about this as if we build multiple models and then aggregate their results in some way.

## Why would we build an "ensemble model?"

Our goal is to estimate $f$, the true function. (Think about $f$ as the **true process** that dictates Ames housing prices.)

We can come up with different hypotheses $h_1$, $h_2$, and so on to get as close to $f$ as possible. (Think about $h_1$ as the model you built to predict $f$, think of $h_2$ as the model your neighbor built to predict $f$, and so on.)

![](./assets/Ensemble.png)


### (Advanced) Three Benefits: Statistical, Computational, Representational
- The **statistical** benefit to ensemble methods: By building one model, our predictions are almost certainly going to be wrong. Predictions from one model might overestimate housing prices; predictions from another model might underestimate housing prices. By "averaging" predictions from multiple models, we'll see that we can often cancel our errors out and get closer to the true function $f$.
- The **computational** benefit to ensemble methods: It might be impossible to develop one model that globally optimizes our objective function. (Remember that CART reach locally-optimal solutions that aren't guaranteed to be the globally-optimal solution.) In these cases, it may be **impossible** for one CART to arrive at the true function $f$. However, generating many different models and averaging their predictions may allow us to get results that are closer to the global optimum than any individual model.
- The **representational** benefit to ensemble methods: Even if we had all the data and all the computer power in the world, it might be impossible for one model to **exactly** equal $f$. For example, a linear regression model can never model a relationship where a one-unit change in $X$ is associated with some *different* change in $Y$ based on the value of $X$. All models have some shortcomings. (See [the no free lunch theorems](https://en.wikipedia.org/wiki/No_free_lunch_in_search_and_optimization).) While individual models have shortcomings, by creating multiple models and aggregating their predictions, we can actually create predictions that represent something that one model cannot ever represent.

We can summarize this as the **wisdom of the crowd**.

## Wisdom of the Crowd: Guess the weight of Penelope

![](./assets/penelope.jpg)

[Image source: https://www.npr.org](https://www.npr.org/sections/money/2015/07/17/422881071/how-much-does-this-cow-weigh)

## Ensemble predictions

Let's mimic the "wisdom of the crowd" by creating several decision trees and averaging their predictions on the test set.

In [ ]:
# What is this line doing?
predictions = pd.DataFrame(index=X_test.index)

In [ ]:
# Generate ten decision trees.

    # Instantiate decision trees.

    # Fit to our training data.

    
    # Put predictions in dataframe.
    predictions[f'Tree {i}'] = t.predict(X_test)

predictions

In [ ]:
# Generate aggregated predicted probabilities.
probs = 

In [ ]:
# Check out probs.


In [ ]:
accuracy_score(y_test, (probs > .5).astype(int))

## Ensemble evaluation?

<details><summary>Why didn't our score improve?</summary>

- Because we fit the same model on the same data ten times!
</details>

## Bootstrapping

Let's get started actually making ensemble predictions. However, in order to do that, we'll need to introduce the idea of bootstrapping, or **random sampling with replacement.**

### Summary
When bootstrapping in order to fit multiple estimators, we want to:
- Take a sample of size $n$.
- With replacement.
- From our original data.

<details><summary>Why do you think we want to take a sample of size n?</summary>
    
- Because we want our estimators to be fit on data of the same size!
- If our original data had $n = 1,000$ and we bootstrapped a sample of size 50 to fit an estimator on, our estimator fit on size 50 will probably look very, very different from an estimator fit on size 1,000.
</details>

<details><summary>Why do you think we want to sample with replacement?</summary>
    
- If we didn't sample with replacement, we'd just get identical samples of size $n$. (These would be copies of our original data!)
</details>

Boostrapping with `pandas`:

## Bagging: Bootstrap Aggregating

As we have seen, decision trees are very powerful machine learning models. However, decision trees have some limitations. In particular, trees that are grown very deep tend to learn highly irregular patterns (a.k.a. they overfit their training sets). 

Bagging (bootstrap aggregating) mitigates this problem by exposing different trees to different sub-samples of the training set.

The process for creating bagged decision trees is as follows:
1. From the original data of size $n$, bootstrap $k$ samples each of size $n$ (with replacement!).
2. Build a decision tree on each bootstrapped sample.
3. Make predictions by passing a test observation through all $k$ trees and developing one aggregate prediction for that observation.

![](./assets/Ensemble.png)

### What do you mean by "aggregate prediction?"
As with all of our modeling techniques, we want to make sure that we can come up with one final prediction for our observation. (Building 1,000 trees and coming up with 1,000 predictions for one observation probably wouldn't be very helpful.)

Suppose we want to predict whether or not a Reddit post is going to go viral, where `1` indicates viral and `0` indicates non-viral. We build 100 decision trees. Given a new Reddit post labeled `X_test`, we pass these features into all 100 decision trees.
- 70 of the trees predict that the post in `X_test` will go viral.
- 30 of the trees predict that the post in `X_test` will not go viral.

**`.predict_proba(X_test)` to do?**
<details><summary>What might you expect .predict(X_test) do?</summary>

- `.predict(X_test)` should output a 1, predicting that the post will go viral.

</details>

<details><summary>What might you expect .predict_proba(X_test) do?</summary>

- `.predict_proba(X_test)` should output 0.7, indicating the probability of the post going viral is 70%.
</details>


## Bagging Classifier using a `for` loop

In the cell below, we'll create an ensemble of trees like before, except this time we'll train each tree to a **bootstrapped** sample of the training data.

In [ ]:
# Instantiate dataframe.
predictions = pd.DataFrame(index=X_test.index)

# Generate ten decision trees.
for i in range(10):
    
    # Bootstrap X data.
    # Should we add a random seed?
    X_sample = 
    
    # Get y data that matches the X data.
    y_sample =
    
    # Instantiate decision trees.
    t = DecisionTreeClassifier()
    
    # Fit to our training data.
    t.fit(X_train, y_train)
    
    # Put predictions in dataframe.
    predictions[f'Tree {i}'] = t.predict(X_test)

predictions.head(20)

In [ ]:
# Generate aggregated predicted probabilities.
predictions.mean(axis='columns')

In [ ]:
# Generate aggregated predicted probabilities as probs.
probs = predictions.mean(axis='columns')

In [ ]:
accuracy_score(y_test, (probs >= .5).astype(int))

## Bagging Classifier using `sklearn`

[BaggingClassifier Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html)

In the cell below, create and score instance of `BaggingClassifier` on the test set. You should get a similar score to the one in the previous step.

In [ ]:
# Instantiate BaggingClassifier.
bag =

# Fit BaggingClassifier.


# Score BaggingClassifier.
